# Predictive task description

Here are the steps:

 

    1. Prepare predictive model(s) of default. Default client is defined as one with loan_status variable taking on the following levels:
    Charged off
    Default
    Does not meet the credit policy. Status: Charged Off
    Late (31-120 days)

    2. Present a few competing predictive models and select one of them. Explain criteria for the selection.
    
    3. Calculate performance metrics you find important in this specific context, but also provide necessarily ROC graphs and AUC for comparison purposes. Metrics should be based on testing set, which should consist 30% randomly selected clients from the original dataset.
    
    4. If necessary, make appropriate variables' pre-processing.
    
    5. Send us a programming code with comments and the report/presentation of main results.

 

We hope you will enjoy solving the task! Good luck!

# PreProcessing, cleaning, feature generating

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 500)

In [3]:
# file paths
raw_data_path = os.path.join(os.path.pardir,'data','raw')
loan_data_path = os.path.join(raw_data_path, 'loan.csv')

In [4]:
dateColumns = ['issue_d', 'earliest_cr_line','next_pymnt_d','last_pymnt_d', 'last_credit_pull_d']

In [5]:
df = pd.read_csv(loan_data_path, low_memory=False, parse_dates=dateColumns, infer_datetime_format=True)

In [6]:
df.shape

(26621, 74)

In [7]:
Default = ['Charged Off','Default','Does not meet the credit policy. Status:Charged Off','Late (31-120 days)']
df['Default'] = 0
df.loc[df['loan_status'].isin(Default), 'Default'] = 1

In [8]:
Active = ['Current','Late (16-30 days)','Issued','In Grace Period']
Paid = ['Fully Paid','Does not meet the credit policy. Status:Fully Paid']
Default = ['Charged Off','Default','Does not meet the credit policy. Status:Charged Off','Late (31-120 days)']
df['loan_status_type'] = 0
df.loc[df['loan_status'].isin(Active), 'loan_status_type'] = 'active'
df.loc[df['loan_status'].isin(Paid), 'loan_status_type'] = 'paid'
df.loc[df['loan_status'].isin(Default), 'loan_status_type'] = 'default'

In [9]:
df['issue_year']=df['issue_d'].dt.year
df['issue_month']=df['issue_d'].dt.month

# ====================================
    which one to drop?

drop with many NuLLs

In [10]:
dropNull = df.isnull().sum().sort_values(ascending=False)/len(df)
columns_to_drop = list(dropNull[dropNull>0.5].index)
df.drop(columns_to_drop, axis=1, inplace=True)

drop with small information and text descriptions

In [11]:
columns_to_drop=[]
columns_to_drop=['policy_code','pymnt_plan','url','member_id','application_type','acc_now_delinq','emp_title','zip_code','title']
df.drop(columns_to_drop, axis=1, inplace=True)

In [12]:
df.shape

(26621, 48)

# ====================================

Transforming objects to int

In [13]:
df['term']=df['term'].str[1:3].astype(int)

In [14]:
dict_emp_length = {'10+ years':10, '2 years':2, '< 1 year':0, '3 years':3, '1 year':1, '5 years':5,
       '4 years':4, 'n/a':0, '7 years':7, '8 years':8, '6 years':6, '9 years':9}
df["emp_length"].replace(dict_emp_length, inplace=True)

In [15]:
dict_status_type = {'active':1, 'paid':2, 'default':0}
df["loan_status_type"].replace(dict_status_type, inplace=True)

In [16]:
dict_grade = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7}
df["grade"].replace(dict_grade, inplace=True)

In [17]:
dict_home_ownership = {'MORTGAGE':2, 'RENT':1, 'OWN':3, 'OTHER':0, 'NONE':0, 'ANY':0}
df["home_ownership"].replace(dict_home_ownership, inplace=True)

In [18]:
dict_initial_list_status = {'f':0, 'w':1}
df["initial_list_status"].replace(dict_initial_list_status, inplace=True)

In [19]:
dict_verification_status = {'Source Verified':1, 'Verified':2, 'Not Verified':0}
df["verification_status"].replace(dict_verification_status, inplace=True)

In [20]:
dict_purpose = {'car':1, 'credit_card':2, 'debt_consolidation':3, 'educational':4,
       'home_improvement':5, 'house':6, 'major_purchase':7, 'medical':8, 'moving':9,
       'other':0, 'renewable_energy':10, 'small_business':11, 'vacation':12, 'wedding':13}
df["purpose"].replace(dict_purpose, inplace=True)

In [21]:
dict_sub_grade = {'A1':11, 'A2':12, 'A3':13, 'A4':14, 'A5':15, 'B1':21, 'B2':22, 'B3':23, 'B4':24, 'B5':25, 'C1':31, 'C2':32,'C3':33, 'C4':34, 'C5':35, 'D1':41, 'D2':42, 'D3':43, 'D4':44, 'D5':45, 'E1':51, 'E2':52, 'E3':53, 'E4':54, 'E5':55, 'F1':61, 'F2':62, 'F3':63, 'F4':64, 'F5':65, 'G1':71, 'G2':72, 'G3':73, 'G4':74, 'G5':75}
df["sub_grade"].replace(dict_sub_grade, inplace=True)

In [22]:
state_default = pd.DataFrame(df.loc[df['Default']==1,'addr_state'].value_counts()).reset_index()
state_default.columns = ['addr_state', 'noDefault']
state = pd.DataFrame(df['addr_state'].value_counts()).reset_index()
state.columns = ['addr_state', 'no']
state_pct = pd.merge(state, state_default, on='addr_state', how='left')
state_pct['state_pct_default']=state_pct['noDefault']/state_pct['no']
state_pct.fillna(0,inplace=True)
state_pct.drop(['no','noDefault'],axis=1,inplace=True)
df = pd.merge(df, state_pct, on='addr_state', how='left')

There are 17586 records which don't have any payment --> last_pymnt_d = issue_d

In [23]:
df.loc[df['last_pymnt_d'].isnull(),'last_pymnt_d'] = df['issue_d']

In [24]:
td = df['last_pymnt_d']-df['issue_d']
df['time_pct_paid']=td.dt.days/(df['term']*365.25/12)

In [25]:
df.shape

(26621, 50)

There are 29 records which don't have credit line --> earliest_cr_line = issue_d

In [26]:
df.loc[df['earliest_cr_line'].isnull(),'earliest_cr_line'] = df['issue_d']

In [27]:
td = df['issue_d']-df['earliest_cr_line']
df['cr_line_days']=td.dt.days

There are 53 records which don't have last credit pull date--> last_credit_pull_d = issue_d

In [28]:
df.loc[df['last_credit_pull_d'].isnull(),'last_credit_pull_d'] = df['issue_d']

In [29]:
td = df['last_credit_pull_d']-df['last_pymnt_d']
df['pull_minus_last_pymnt']=td.dt.days

There are 4 records which don't have annual income--> annual_inc = 0

In [30]:
df.loc[df['annual_inc'].isnull(),'annual_inc'] = 0

no longer needed

In [31]:
df.drop(dateColumns, axis=1, inplace=True)

fill NA/Nulls with mode

In [32]:
with_null_val = ['total_rev_hi_lim', 'tot_cur_bal', 'tot_coll_amt','revol_util','collections_12_mths_ex_med', 'delinq_2yrs','inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc']

In [33]:
for col in with_null_val:
    df[col].fillna(df[col].mode()[0],inplace=True)

In [34]:
df.shape

(26621, 47)

In [35]:
object_col = ['loan_status','loan_status_type','addr_state']
df.drop(object_col, axis=1, inplace=True)

# ===========================================

In [36]:
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,Default,issue_year,issue_month,state_pct_default,time_pct_paid,cr_line_days,pull_minus_last_pymnt
0,22432240,7025.0,7025.0,7025.0,36,16.99,250.43,4,43,7,1,33000.0,1,3,26.12,0.0,3.0,20.0,0.0,9949.0,58.9,30.0,0,4146.22,4146.22,4257.310000,4257.31,2878.78,1378.53,0.0,0.0,0.0,250.43,0.0,0.0,21182.0,16900.0,0,2014,7,0.085921,0.472736,6117,31
1,21461200,27000.0,27000.0,27000.0,60,14.99,642.19,3,35,8,1,65000.0,1,0,25.84,0.0,2.0,14.0,0.0,34865.0,49.5,23.0,1,21264.62,21264.62,10917.230000,10917.23,5735.38,5181.85,0.0,0.0,0.0,642.19,0.0,0.0,53409.0,70500.0,0,2014,7,0.065719,0.283641,8004,31
2,7685410,6550.0,6550.0,6550.0,36,19.52,241.83,4,44,10,2,60389.0,2,3,3.95,0.0,4.0,4.0,0.0,1655.0,31.8,24.0,1,0.00,0.00,6962.840000,6962.84,6550.00,412.84,0.0,0.0,0.0,6237.35,0.0,0.0,161915.0,5200.0,0,2013,10,0.049180,0.112252,3775,699
3,14859139,10000.0,10000.0,10000.0,36,8.90,317.54,1,15,1,1,50000.0,0,3,12.41,0.0,0.0,9.0,1.0,3912.0,23.0,20.0,0,4773.83,4773.83,6350.800000,6350.80,5226.17,1124.63,0.0,0.0,0.0,317.54,0.0,0.0,6113.0,17000.0,0,2014,5,0.053097,0.556696,8278,0
4,1904812,5000.0,5000.0,5000.0,36,6.03,152.18,1,11,8,2,65000.0,1,0,6.11,0.0,0.0,13.0,0.0,4956.0,8.4,34.0,0,0.00,0.00,5377.173815,5377.17,5000.00,377.17,0.0,0.0,0.0,2487.85,0.0,0.0,257401.0,58700.0,0,2012,11,0.064033,0.553958,9893,0


In [37]:
# file paths
processed_data_path = os.path.join(os.path.pardir,'data','processed')
loan_preprocessed = os.path.join(processed_data_path, 'loan_preprocessed.csv')

In [38]:
df.to_csv(loan_preprocessed,index=False)

In [39]:
df = pd.read_csv(loan_preprocessed, low_memory=False)

In [40]:
y = df['Default']
X = df.drop(['Default'], axis=1)

In [41]:
# file pahts
X_path = os.path.join(processed_data_path, 'X.csv')
y_path = os.path.join(processed_data_path, 'y.csv')
X_train_path = os.path.join(processed_data_path, 'X_train.csv')
y_train_path = os.path.join(processed_data_path, 'y_train.csv')
X_test_path = os.path.join(processed_data_path, 'X_test.csv')
y_test_path = os.path.join(processed_data_path, 'y_test.csv')

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
X.to_csv(X_path, index=False)
y.to_csv(y_path, index=False)
X_train.to_csv(X_train_path, index=False)
y_train.to_csv(y_train_path, index=False)
X_test.to_csv(X_test_path, index=False)
y_test.to_csv(y_test_path, index=False)

In [44]:
print(y.value_counts(), y.mean())

0    24762
1     1859
Name: Default, dtype: int64 0.0698320874497577


# ===========================================